In [ ]:
from pybbn.graph.dag import Bbn
from pybbn.graph.edge import Edge, EdgeType
from pybbn.graph.node import BbnNode
from pybbn.graph.variable import Variable
from pybbn.pptc.inferencecontroller import InferenceController
from pybbn.graph.jointree import EvidenceBuilder

def getEnfermedades():
    #Variables
    var_tos = Variable(0, 'tos', ['si', 'no']) 
    var_gripe = Variable(1, 'gripe', ['si', 'no'])
    var_fiebre = Variable(2, 'fiebre', ['si', 'no'])
    var_tuberculosis = Variable(3, 'propenso a tuberculosis', ['si', 'no'])
    var_bronquitis = Variable(4, 'propenso a bronquitis', ['si', 'no'])
    var_covid = Variable(5, 'covid', ['si', 'no'])
    

    #Tablas de probabilidad
    tabla_tos = [0.60,0.40 ] 
    tabla_gripe = [0.60, 0.40] #tendremos que agregar con cuidado el + primero y el - despues
    tabla_fiebre = [0.60,0.40]
    tabla_tuberculosis = [0.43,0.57,  0.45,0.55]  #0.40,0.60,  0.48,0.52,  0.39,0.61,  0.47,0.53,  0.42,0.68,  0.49,0.51,  0.38,0.62,  0.41,0.59,  0.37,0.63,  0.44,0.56,  0.36,0.64,  0.43,0.57,  0.41,0.59,  0.46,0.54]
    tabla_bronquitis = [0.44,0.56,  0.46,0.54,  0.41,0.59,  0.49,0.51,  0.40,0.60,  0.48,0.52,  0.43,0.57,  0.50,0.50,  0.41,0.59,  0.41,0.56,  0.39,0.61,  0.47,0.53,  0.38,0.62,  0.45,0.55,  0.42,0.58,  0.49,0.51]
    tabla_covid = [0.45,0.55,  0.47,0.53,  0.42,0.58,  0.50,0.50,  0.41,0.59,  0.49,0.51,  0.44,0.56,  0.51,0.49,  0.42,0.58,  0.45,0.55,  0.40,0.60,  0.48,0.52,  0.39,0.61,  0.46,0.54,  0.43,0.57,  0.50,0.50]
      



    #Nodos
    nodotos = BbnNode(var_tos, tabla_tos)
    nodogripe = BbnNode(var_gripe, tabla_gripe)
    nodofiebre = BbnNode(var_fiebre, tabla_fiebre)
    nodotuberculosis= BbnNode(var_tuberculosis, tabla_tuberculosis)
    nodobronquitis = BbnNode(var_bronquitis, tabla_bronquitis)
    nodocovid = BbnNode(var_covid, tabla_covid)
    
    

    #Modelo
    bayesiana = Bbn().add_node(nodotos).add_node(nodogripe).add_node(nodotuberculosis).add_node(nodobronquitis).add_node(nodocovid).add_node(nodofiebre)\

    bayesiana.add_edge(Edge(nodotos, nodocovid, EdgeType.DIRECTED)) #agregamos la conexion de los nodos dependiendo cual conecta con cual
    bayesiana.add_edge(Edge(nodotos, nodotuberculosis, EdgeType.DIRECTED))
    bayesiana.add_edge(Edge(nodotos, nodobronquitis, EdgeType.DIRECTED))
    
    bayesiana.add_edge(Edge(nodofiebre, nodocovid, EdgeType.DIRECTED))
    bayesiana.add_edge(Edge(nodofiebre, nodobronquitis, EdgeType.DIRECTED))
    
    bayesiana.add_edge(Edge(nodogripe, nodobronquitis, EdgeType.DIRECTED))
    bayesiana.add_edge(Edge(nodogripe, nodocovid, EdgeType.DIRECTED))
    
    #bayesiana.add_edge(Edge(nodocovid, nodotuberculosis, EdgeType.DIRECTED))
    #bayesiana.add_edge(Edge(nodocovid, nodobronquitis, EdgeType.DIRECTED))
    
    
    
  
    return bayesiana

def getMotorInferencia():
    red = getEnfermedades()
    control_inferencia = InferenceController.apply(red)
    return control_inferencia

def printPosteriors(join_tree):
    for node, posteriors in join_tree.get_posteriors().items():
        p = ', '.join([f'{val}={prob:.5f}' for val, prob in posteriors.items()])
        print(f'{node} : {p}')

In [ ]:
#verificamos las inferencias
modelo=getMotorInferencia()



In [ ]:
#Ejemplo: Si una persona tiene gripe, tos, pero no tiene COVID, ¿cuál es la probabilidad de bronquitis y tuberculosis?

modelo.unobserve_all() #reiniciamos la wea

nodotos = modelo.get_bbn_node_by_name('tos')
evb = (EvidenceBuilder() \
    .with_node(nodotos) \
    .with_evidence('si',1.0) \
    .build())


nodogripe = modelo.get_bbn_node_by_name('gripe')
evc = (EvidenceBuilder() \
    .with_node(nodogripe) \
    .with_evidence('si', 1.0) \
    .build())


nodocovid = modelo.get_bbn_node_by_name('covid')
eve = (EvidenceBuilder() \
    .with_node(nodocovid) \
    .with_evidence('no', 1.0) \
    .build())

modelo.set_observation(evb)
modelo.set_observation(evc)
modelo.set_observation(eve)

printPosteriors(modelo)


tos : si=1.00000, no=0.00000
propenso a tuberculosis : si=0.43000, no=0.57000
gripe : si=1.00000, no=0.00000
fiebre : si=0.58719, no=0.41281
propenso a bronquitis : si=0.42762, no=0.57238
covid : si=0.00000, no=1.00000


In [33]:
#Ejemplo: Si una persona tiene tos, ya tiene tuberculosis, pero no tiene COVID. ¿Cuál es su probabilidad de tener bronquitis y tener gripe?

modelo.unobserve_all() #reiniciamos la wea

nodotos = modelo.get_bbn_node_by_name('tos')
evb = (EvidenceBuilder() \
    .with_node(nodotos) \
    .with_evidence('si',1.0) \
    .build())


nodocovid = modelo.get_bbn_node_by_name('covid')
evd = (EvidenceBuilder() \
    .with_node(nodocovid) \
    .with_evidence('no', 1.0) \
    .build())


nodotuberculosis = modelo.get_bbn_node_by_name('propenso a tuberculosis')
eve = (EvidenceBuilder() \
    .with_node(nodotuberculosis) \
    .with_evidence('si', 1.0) \
    .build())

modelo.set_observation(evb)
modelo.set_observation(evd)
modelo.set_observation(eve)

printPosteriors(modelo)

tos : si=1.00000, no=0.00000
propenso a tuberculosis : si=1.00000, no=0.00000
gripe : si=0.61940, no=0.38060
fiebre : si=0.59735, no=0.40265
propenso a bronquitis : si=0.44435, no=0.55565
covid : si=0.00000, no=1.00000


In [35]:
#Ejemplo: Si una persona no tiene tos y si tiene gripe, ¿Cuál es la probabilidad de que tenga covid y fiebre?

modelo.unobserve_all() #reiniciamos la wea

nodotos = modelo.get_bbn_node_by_name('tos')
evb = (EvidenceBuilder() \
    .with_node(nodotos) \
    .with_evidence('no',1.0) \
    .build())


nodogripe = modelo.get_bbn_node_by_name('gripe')
evc = (EvidenceBuilder() \
    .with_node(nodogripe) \
    .with_evidence('si', 1.0) \
    .build())


modelo.set_observation(evb)
modelo.set_observation(evc)


printPosteriors(modelo)

tos : si=0.00000, no=1.00000
propenso a tuberculosis : si=0.45000, no=0.55000
gripe : si=1.00000, no=0.00000
fiebre : si=0.60000, no=0.40000
propenso a bronquitis : si=0.41200, no=0.58800
covid : si=0.42200, no=0.57800


In [ ]:
#Ejemplo: Si una persona no tiene fiebre pero si tiene covid cual es la probabilidad de padecer bronquitis y tuberculosis?

modelo.unobserve_all() #reiniciamos la wea

nodofiebre = modelo.get_bbn_node_by_name('fiebre')
evb = (EvidenceBuilder() \
    .with_node(nodofiebre) \
    .with_evidence('no',1.0) \
    .build())


nodocovid = modelo.get_bbn_node_by_name('covid')
evc = (EvidenceBuilder() \
    .with_node(nodocovid) \
    .with_evidence('si', 1.0) \
    .build())


modelo.set_observation(evb)
modelo.set_observation(evc)


printPosteriors(modelo)

tos : si=0.59214, no=0.40786
propenso a tuberculosis : si=0.43816, no=0.56184
gripe : si=0.58722, no=0.41278
fiebre : si=0.00000, no=1.00000
propenso a bronquitis : si=0.44951, no=0.55049
covid : si=1.00000, no=0.00000


In [ ]:
#Ejemplo: Si una persona no tiene fiebre, ni gripe pero si tiene tos,cual es su probabilidad de tener covid?

modelo.unobserve_all() #reiniciamos la wea

nodofiebre = modelo.get_bbn_node_by_name('fiebre')
evb = (EvidenceBuilder() \
    .with_node(nodofiebre) \
    .with_evidence('no',1.0) \
    .build())


nodotos = modelo.get_bbn_node_by_name('tos')
evd = (EvidenceBuilder() \
    .with_node(nodotos) \
    .with_evidence('si', 1.0) \
    .build())


nodogripe = modelo.get_bbn_node_by_name('gripe')
eve = (EvidenceBuilder() \
    .with_node(nodogripe) \
    .with_evidence('no', 1.0) \
    .build())

modelo.set_observation(evb)
modelo.set_observation(evd)
modelo.set_observation(eve)

printPosteriors(modelo)

tos : si=1.00000, no=0.00000
propenso a tuberculosis : si=0.43000, no=0.57000
gripe : si=0.00000, no=1.00000
fiebre : si=0.00000, no=1.00000
propenso a bronquitis : si=0.49000, no=0.51000
covid : si=0.50000, no=0.50000
